In [62]:
import ee
import geemap as emap

In [63]:
# Initialize ee
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWiYZQbjwDR6lihM_UYPhRtJEHZnj_BKyYBLON60bXGku0MkG8TSw-U

Successfully saved authorization token.


In [65]:
Map=emap.Map()

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# <font color="blue"> 1. Create a image collection </font>

In [67]:
# Create arbitrary constant images.
constant1 = ee.Image(1).rename("B1") # create a constant image and rename it as B1
constant2 = ee.Image(2).rename("B2")
# Get info of image1
constant1.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

**Create a image collection from various single images/bands**

In [68]:
# Create a collection by giving a list to the constructor.
collectionFromConstructor = ee.ImageCollection([constant1, constant2])
print('collectionFromConstructor: ', collectionFromConstructor.getInfo())

collectionFromConstructor:  {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '1'}}]}


In [69]:
# Alterantively, create a collection with fromImages().
collectionFromImages = ee.ImageCollection.fromImages(
  [ee.Image(3.5), ee.Image(4)])
print('collectionFromImages: ', collectionFromImages.getInfo())

collectionFromImages:  {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 3.5, 'max': 3.5}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 4, 'max': 4}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '1'}}]}


**We can merge two or more collection**

In [70]:
# Merge two collections.
mergedCollection = collectionFromConstructor.merge(collectionFromImages)
print('mergedCollection: ', mergedCollection.getInfo())

mergedCollection:  {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '1_0'}}, {'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '1_1'}}, {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 3.5, 'max': 3.5}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '2_0'}}, {'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 4, 'max': 4}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '2_1'}}]}


# <font color="blue"> Select image Collection </font>

In [71]:
VN=ee.FeatureCollection("users/havantuyen/VN_Map")
Map.centerObject(VN,5)
Map.addLayer(VN,{},"Vietnam")
Map

Map(bottom=1851.0, center=[16.453064229938988, 106.45862984398038], controls=(WidgetControl(options=['position…

In [72]:
# Load a Landsat 8 ImageCollection for VN for 2020
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2020-01-01', '2020-12-31').filterBounds(VN).\
filterMetadata("CLOUD_COVER","less_than",30) # Get only images with less than 30% of cloud

# Get the number of images.
count = landsat8.size()
print(f" A total of {count.getInfo()} images")

 A total of 308 images


In [73]:
# MODIS temperature for 2020
modis=ee.ImageCollection("MODIS/006/MOD11A1").filterDate('2020-01-01', '2020-12-31').filterBounds(VN).select("LST_Day_1km")

# Get the number of images.
count = modis.size()
print(f"{count.getInfo()} images")

365 images


In [74]:
# Get the date range of images in the collection.
range = landsat8.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', ee.Date(range.get('min')).getInfo(), ee.Date(range.get('max')).getInfo())

Date range:  {'type': 'Date', 'value': 1577936135573} {'type': 'Date', 'value': 1609298419022}


In [75]:
# Better way of extracting start and end time
landsat8.get("date_range").getInfo()

[1365638400000, 1633564800000]

In [76]:
# get human readbale date format.
range1 = landsat8.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start_date = ee.Date(range1.get('min')).format("YYYY-MM-dd")
end_date = ee.Date(range1.get('max')).format("YYYY-MM-dd")
print('Date range: ', start_date.getInfo(), end_date.getInfo())

Date range:  2020-01-02 2020-12-30


In [77]:
# Get statistics for a property of the images in the collection.
sunStats = landsat8.aggregate_stats('RADIANCE_ADD_BAND_3') # it aggregates all images and calculate statistics for a given property
print('Sun elevation statistics: ', sunStats.getInfo())

Sun elevation statistics:  {'max': -57.31108856201172, 'mean': -59.41173877963772, 'min': -61.275508880615234, 'sample_sd': 1.4418822426544458, 'sample_var': 2.0790244016822137, 'sum': -18298.815544128418, 'sum_sq': 1087802.7095758482, 'total_count': 308, 'total_sd': 1.4395396217040963, 'total_var': 2.0722743224559728, 'valid_count': 308, 'weight_sum': 308, 'weighted_sum': -18298.815544128418}


In [78]:
# List a value of a property for all images in the collection.
clouds = landsat8.aggregate_array('CLOUD_COVER') # List cloud cover percentages of all images in the collection
print('Cloud covers: ', clouds.getInfo())

Cloud covers:  [8.390000343322754, 11.920000076293945, 0.36000001430511475, 11, 3.8499999046325684, 5.809999942779541, 2.9700000286102295, 10.680000305175781, 9.170000076293945, 7.369999885559082, 26.770000457763672, 4.849999904632568, 5.099999904632568, 12.619999885559082, 7.440000057220459, 11.979999542236328, 22.559999465942383, 7.730000019073486, 9.40999984741211, 16.190000534057617, 11.520000457763672, 4.71999979019165, 12.229999542236328, 16.760000228881836, 5.889999866485596, 3.630000114440918, 5.619999885559082, 7.340000152587891, 7.300000190734863, 23.829999923706055, 15.130000114440918, 25.829999923706055, 13.850000381469727, 6.119999885559082, 6.389999866485596, 19.510000228881836, 3.25, 3.2699999809265137, 8.640000343322754, 5.96999979019165, 4.099999904632568, 21.219999313354492, 10.100000381469727, 6.96999979019165, 9.649999618530273, 2.609999895095825, 7.800000190734863, 6.760000228881836, 16.56999969482422, 23.84000015258789, 23.020000457763672, 29.59000015258789, 7.159

In [79]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(landsat8.sort('CLOUD_COVER').first())
print('\nCloud cover: ', image.get("CLOUD_COVER").getInfo())


Cloud cover:  0


In [80]:
# Limit the collection to the 10 images.
recent = landsat8.sort('system:time_start', False).limit(10)
images = recent.aggregate_array('system:index').getInfo()
print(" 10 recent images:")
for image in images:
    print(image)

 10 recent images:
LC08_126052_20201230
LC08_128047_20201228
LC08_128046_20201228
LC08_127053_20201221
LC08_128047_20201212
LC08_128046_20201212
LC08_125050_20201207
LC08_125045_20201207
LC08_118054_20201206
LC08_127054_20201205


# 1. Select Multiple Images

##### Filter or get `best` image from a ImageCollection

In [33]:
# Filter date, bounds and sort by cloud cover
## Define the region of interest
point=ee.Geometry.Point(105.46,21.41) # Hanoi, Vietnam
## Filter by collection
LS8=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2014-01-01","2015-01-01").\
filterBounds(point).sort("CLOUD_COVER") # 2014

In [37]:
# Print out the number of images 
print("a total of",LS8.size().getInfo(),"images")


a total of 17 images


In [46]:
# Visualizing the above image
vis={"bands":["B4","B3","B2"],"min":0.046, "max":0.389}
# Center the map
Map.centerObject(point,10)
# Plot the image
Map.addLayer(image, vis, "Image 2014")
Map

Map(bottom=115406.0, center=[21.41, 105.46000000000001], controls=(WidgetControl(options=['position', 'transpa…

In [ ]:
# Investigate the image property
image.getInfo()

**We can add property to an existing image**

In [44]:
# Set property for image
date_code=ee.Date(image.get('system:time_start')).format("dd-MM-YYYY")

# date_code.getInfo()

addedProperty=image.set({"Date": date_code,"City":"Ha Noi"})

print(addedProperty.get("City").getInfo())

Ha Noi


##### Write a function to select the best image by each year from 2015 to 2020

In [47]:
def bestImage(year):
    start_year=ee.Date.fromYMD(year, 1,1)
    end_year=start_year.advance(1,"year") # increase 1 year every iteration
    image=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate(start_year,end_year).\
filterBounds(point).sort("CLOUD_COVER").first()
    return image
years=ee.List.sequence(2014,2020)

myImage=years.map(bestImage)
# See the number of images
myImage.size().getInfo()

7

In [48]:
# Combine to make an image collection
collection=ee.ImageCollection(myImage)
# See cloud cover each image
for i in sorted(collection.aggregate_array("CLOUD_COVER").getInfo()):
    print("Cloud Cover: {:.3} %".format(i))

Cloud Cover: 0.12 %
Cloud Cover: 0.83 %
Cloud Cover: 1.53 %
Cloud Cover: 5.54 %
Cloud Cover: 8.37 %
Cloud Cover: 19.1 %
Cloud Cover: 25.1 %


In [49]:
# Visualize each image
for index in [0,1,2,3,4,5,6]:
    image=ee.Image(myImage.get(index))
    layerName="Image "+ str(years.getInfo()[index])
    Map.addLayer(image, vis, layerName, False)
Map

Map(bottom=115406.0, center=[21.41, 105.46000000000001], controls=(WidgetControl(options=['position', 'transpa…

##### Writing a function to select monthly images 

In [ ]:
years=range(2015,2021)
months=range(1,13)
import datetime as dt
for year in years:
    for month in months:
        start=ee.Date.fromYMD(year,month,1)
        end=start.advance(1, "month")
#         print("Starting month:",start.get("month").getInfo(),"-", end.get("month").getInfo())
# It take some time

## Display property as our desire

In [61]:
# get all properties information
keys=image.getInfo()

for key in keys:
    if key=="properties":
        for i in keys[key]:
            print(i,":",keys[key][i])

RADIANCE_MULT_BAND_5 : 0.005916700232774019
RADIANCE_MULT_BAND_6 : 0.0014714000280946493
RADIANCE_MULT_BAND_3 : 0.011466000229120255
RADIANCE_MULT_BAND_4 : 0.00966859981417656
RADIANCE_MULT_BAND_1 : 0.012151000089943409
RADIANCE_MULT_BAND_2 : 0.012443000450730324
K2_CONSTANT_BAND_11 : 1201.1441650390625
K2_CONSTANT_BAND_10 : 1321.078857421875
system:footprint : {'type': 'LinearRing', 'coordinates': [[106.19853233208539, 20.614781976648437], [106.1991383755885, 20.615902250262256], [106.22327794516717, 20.716529413691653], [106.25403100186922, 20.84461777695151], [106.31506237316437, 21.098238400091283], [106.35810525150184, 21.276752088691048], [106.42216046442657, 21.541764324387373], [106.58263092472738, 22.202740623662542], [106.61798347299015, 22.34859547370889], [105.22996972738969, 22.634444961613006], [104.81971550744727, 22.71642664247542], [104.81412529374815, 22.71745718806048], [104.81234222728955, 22.71163970233053], [104.73975009574556, 22.400542446447353], [104.4494997163